# Interactive Demo for Fors2 with Spectro Photometry to Read results



- author (adaptation) : Sylvie Dagoret-Campagne
- creation date : 2023-06-16
- last update : 2023-06-26


https://github.com/bd-j/prospector/blob/main/demo/tutorial.rst

# Visualizing the Results
There are a few basic plotting tools available to do a quick check on the results available in `prospect.io.read_results` and `prospect.utils.plotting`. But first we need to read the output files we made.

## Reading output files
Reading our results from our HDF5 file is straightforward using the `results_from` method.  This returns a "results" dictionary, the `obs` dictionary of data to which the model was fit, and in some situations the `SedModel` object that was used in the fitting.

In this example we will look at the *emcee* results file that we created, but you can do the same for the *dynesty* results file.  The main difference is that the chains for each fit type have a different dimensionality, and that the samples in the *dynesty* chains come with associated **weights**.  The weights and different shapes are handled automatically by the plotting functions included in *prospector*, but custom plotting code should take care of the difference.

In this notebook we have already defined the `model` (and `sps`) object, but in general one will want to recreate it.  *Prospector* allows you to do this by storing the `build_model` and `build_sps` code as text in the HDF5 file -- this is possible with the use of *parameter files* and command line operation.

In [1]:
import numpy as np
import re
import os

In [2]:
import matplotlib.pyplot as plt
from matplotlib import rcParams

from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap

In [3]:
!ls output/spectrophotom

SPEC36	SPEC411  SPEC560


In [4]:
SPECNUM=560
SPECNAME=f"SPEC{SPECNUM}"

In [5]:
output_path=os.path.join("output/spectrophotom",SPECNAME)

In [6]:
all_files = os.listdir(output_path)

In [7]:
all_files = sorted(all_files)
all_files

['run_optimize_SPEC560_spectrophotom_23Jun24-05.02_result.h5',
 'run_optimize_SPEC560_spectrophotom_23Jun25-22.43_result.h5',
 'run_optimize_SPEC560_spectrophotom_23Jun26-11.15_result.h5',
 'run_optimize_dynesty_SPEC560_spectrophotom_23Jun24-05.20_result.h5',
 'run_optimize_dynesty_SPEC560_spectrophotom_23Jun25-23.09_result.h5',
 'run_optimize_dynesty_SPEC560_spectrophotom_23Jun26-11.27_result.h5',
 'run_optimize_emcee_SPEC560_spectrophotom_23Jun24-05.09_result.h5',
 'run_optimize_emcee_SPEC560_spectrophotom_23Jun25-22.52_result.h5',
 'run_optimize_emcee_SPEC560_spectrophotom_23Jun26-11.19_result.h5']

In [8]:
file_idx=5

In [9]:
filename = os.path.join(output_path,all_files[file_idx])
filename

'output/spectrophotom/SPEC560/run_optimize_dynesty_SPEC560_spectrophotom_23Jun26-11.27_result.h5'

In [10]:
test = re.findall("dynesty",filename)
if len(test)>0:
    results_type = "dynesty"
test = re.findall("emcee",filename)
if len(test)>0:
    results_type = "emcee"

In [11]:
results_type

'dynesty'

In [13]:
import prospect.io.read_results as reader
#results_type = "emcee" # | "dynesty"
#results_type = "dynesty" # | "dynesty"
# grab results (dictionary), the obs dictionary, and our corresponding models
# When using parameter files set `dangerous=True`
result, obs, model = reader.results_from(filename, dangerous=True)

#The following commented lines reconstruct the model and sps object, 
# if a parameter file continaing the `build_*` methods was saved along with the results
model = reader.get_model(result)
sps = reader.get_sps(result)

# let's look at what's stored in the `result` dictionary
print(result.keys())

KeyError: 'run_params'

In [ ]:
result['theta_labels']

In [ ]:
result['bestfit']

In [ ]:
obs

In [ ]:
plt.plot(obs['wavelength'],obs["spectrum"],'g-')
plt.plot(obs['phot_wave'],obs["maggies"],'or')

In [ ]:
result['theta_labels']

In [ ]:
result['bestfit']['parameter']

In [ ]:
best = result['bestfit']

In [ ]:
all_param_dict = {}
idx=0
for key in result['theta_labels']:
    all_param_dict[key] = result['bestfit']['parameter'][idx]
    idx+=1
all_param_dict    

In [ ]:
model.fixed_params

In [ ]:
model.free_params

In [ ]:
#result['run_params']

In [ ]:
for the_dict in result['model_params']:
    if not the_dict['isfree']:
        all_param_dict[the_dict['name']] = the_dict['init']

In [ ]:
sps.update(**all_param_dict)

In [ ]:
sps.params

In [ ]:
zred = sps.params['zred']
a= 1+zred

In [ ]:
spectr  = sps.get_spectrum()[0]

In [ ]:
plt.loglog(sps.wavelengths*a,spectr/a,'k-')
plt.axvline(3646,color='k')
plt.plot(obs['wavelength'],obs["spectrum"],'g-',lw=3)
plt.plot(obs['phot_wave'],obs["maggies"],'or',ms=2)

## Plotting parameter traces
To see how our MCMC samples look, we can examine a few traces (that is, the evolution of the parameter value with iteration in the MCMC chain.)  You can use these plots (and the chains more generally) to assess whether the MCMC has converged, or if you need to sample for more iterations.

In [ ]:
if results_type == "emcee":
    chosen = np.random.choice(result["run_params"]["nwalkers"], size=10, replace=False)
    tracefig = reader.traceplot(result, figsize=(20,10), chains=chosen)
else:
    tracefig = reader.traceplot(result, figsize=(20,10))


In [ ]:
# Trace plots
tfig = reader.traceplot(result)
# Corner figure of posterior PDFs
cfig = reader.subcorner(result)


In [ ]:
# Maximum posterior probability sample
imax = np.argmax(result['lnprobability'])
csz = result["chain"].shape
if result["chain"].ndim > 2:
    # emcee
    i, j = np.unravel_index(imax, result['lnprobability'].shape)
    theta_max = result['chain'][i, j, :].copy()
    flatchain = result["chain"].reshape(csz[0] * csz[1], csz[2])
else:
    # dynesty
    theta_max = result['chain'][imax, :].copy()
    flatchain = result["chain"]

# 16th, 50th, and 84th percentiles of the posterior
from prospect.plotting.corner import quantile
weights = result.get("weights", None)
post_pcts = quantile(flatchain.T, q=[0.16, 0.50, 0.84], weights=weights)

In [ ]:
# Plot the stored maximum ln-probability sample
import matplotlib.pyplot as pl

best = result["bestfit"]
a = model.params["zred"] + 1
#pl.plot(a * best["restframe_wavelengths"], best['spectrum'], label="MAP spectrum")
pl.plot(obs["wavelength"], best['spectrum'], label="MAP spectrum")
pl.plot(obs["wavelength"], obs['spectrum'], label="obs spectrum")
if obs['filters'] is not None:
    pwave = [f.wave_effective for f in obs["filters"]]
    pl.loglog(pwave, best['photometry'],'o' ,label="MAP photometry")
    pl.title(best["parameter"])
pl.legend()


In [ ]:
best

## Making a corner plot
Our samples more generally can be shown using a corner/triangle plot.  The `subtriangle()` method below is a very thin wrapper on Dan Foreman-Mackey's **corner.py** code.  We'll overplot the MAP value as blue lines.

In [ ]:
#ind_best = np.argmin([r.cost for r in result])
#print(ind_best)
theta_best = best['parameter']
print(theta_best)

# generate model
prediction = model.mean_model(theta_best, obs=obs, sps=sps)
pspec, pphot, pfrac = prediction


In [ ]:
# maximum a posteriori (of the locations visited by the MCMC sampler)
imax = np.argmax(result['lnprobability'])
if results_type == "emcee":
    i, j = np.unravel_index(imax, result['lnprobability'].shape)
    theta_max = result['chain'][i, j, :].copy()
    thin = 5
else:
    theta_max = result["chain"][imax, :]
    thin = 1

#print('Optimization value: {}'.format(theta_best))
print('MAP value: {}'.format(theta_max))
cornerfig = reader.subcorner(result, start=0, thin=thin, truths=theta_best, 
                             fig=pl.subplots(5,5,figsize=(27,27))[0])

## Look at SEDs and residuals
Finally, let's just take a look at a random model drawn from our chains, and at the highest posterior probability model in the chain.  In this notebook we already have the `sps` object instantiated, but in general we may have to regenerate it based on information stored in the output file using the `prospect.io.read_results.get_sps` method.

In [ ]:
obs = result["obs"]
obs

In [ ]:
a = 1.0 + model.params.get('zred', 0.0) # cosmological redshifting
# photometric effective wavelengths
wphot = obs["phot_wave"]
# spectroscopic wavelengths
if obs["wavelength"] is None:
    # *restframe* spectral wavelengths, since obs["wavelength"] is None
    wspec = sps.wavelengths
    wspec *= a #redshift them
else:
    wspec = obs["wavelength"]

In [ ]:
wspec.shape

In [ ]:
wphot.shape

In [ ]:
run_params = result["run_params"]
run_params

In [ ]:
import matplotlib.pyplot as plt

# randomly chosen parameters from chain
randint = np.random.randint
if results_type == "emcee":
    nwalkers, niter = run_params['nwalkers'], run_params['niter']
    theta = result['chain'][randint(nwalkers), randint(niter)]
else:
    theta = result["chain"][randint(len(result["chain"]))]

# generate models
# sps = reader.get_sps(result)  # this works if using parameter files
mspec, mphot, mextra = model.mean_model(theta, obs, sps=sps)
mspec_map, mphot_map, _ = model.mean_model(theta_max, obs, sps=sps)


# establish bounds
xmin, xmax = np.min(wphot)*0.8, np.max(wphot)/0.8
temp = np.interp(np.linspace(xmin,xmax,10000), wspec, mspec_map)
ymin, ymax = temp.min()*0.8, temp.max()/0.4

# Make plot of data and model
plt.figure(figsize=(16,8))

#plt.loglog(wspec, mspec, label='Model spectrum (random draw)',
#       lw=0.7, color='green', alpha=0.7)
plt.loglog(wspec, mspec_map, label='Model spectrum (MAP)',
       lw=0.7, color='navy', alpha=0.7)
#plt.errorbar(wphot, mphot, label='Model photometry (random draw)',
#         marker='s', markersize=10, alpha=0.8, ls='', lw=3, 
#         markerfacecolor='none', markeredgecolor='green', 
#         markeredgewidth=3)
plt.errorbar(wphot, mphot_map, label='Model photometry (MAP)',
         marker='s', markersize=10, alpha=0.8, ls='', lw=3, 
         markerfacecolor='none', markeredgecolor='blue', 
         markeredgewidth=3)
plt.errorbar(wphot, obs['maggies'], yerr=obs['maggies_unc'], 
         label='Observed photometry', ecolor='red', 
         marker='o', markersize=10, ls='', lw=3, alpha=0.8, 
         markerfacecolor='none', markeredgecolor='red', 
         markeredgewidth=3)

# plot transmission curves
for f in obs['filters']:
    w, t = f.wavelength.copy(), f.transmission.copy()
    t = t / t.max()
    t = 10**(0.2*(np.log10(ymax/ymin)))*t * ymin
    plt.loglog(w, t, lw=3, color='gray', alpha=0.7)

plt.xlabel('Wavelength [A]')
plt.ylabel('Flux Density [maggies]')
plt.xlim([xmin, xmax])
plt.ylim([ymin, ymax])
plt.legend(loc='best', fontsize=20)
plt.tight_layout()
plt.axvline(3646)
#plt.loglog(sps.wavelengths*a,sps.get_spectrum()[0])